In [11]:
# Required imports
import tkinter as tk
import os
import time
from tkinter import ttk
from tkinter import StringVar, Listbox, MULTIPLE
import requests
import pandas as pd
import threading
from threading import Lock
from datetime import datetime
from itertools import product


# Constants for caching
CACHE_EXPIRY_TIME = 3600  # in seconds, 1 hour

# Variable to hold the last update time
last_update_time = {}

sports_to_market_mapping = {
    'mma_mixed_martial_arts': ['h2h'],
    'boxing_boxing': ['h2h'],
    'americanfootball_nfl': ['h2h', 'spreads','totals'],
    'americanfootball_ncaaf': ['h2h', 'spreads'],
    'basketball_nba': ['h2h', 'spreads'],
    'baseball_mlb': ['h2h', 'spreads'],
    'icehockey_nhl': ['h2h', 'spreads',]
}

# List of known bookmakers
known_bookmakers = ['FanDuel', 'BetUS', 'DraftKings', 'SuperBook', 'Caesars', 'LowVig.ag', 'BetOnline.ag', 'Bovada', 'TwinSpires', 'BetRivers', 'Unibet', 'MyBookie.ag', 'PointsBet (US)', 'WynnBET', 'BetMGM', 'Barstool Sportsbook']

# Initialize global variables for API quota
api_requests_used = 0
api_requests_remaining = 0

# Initialize markets as a hyperparameter
markets = ['h2h','spreads']

# Fetch and prepare odds data
def download_odds(sport, api_quota_label, selected_bookmakers='All', selected_markets=None):
    global api_requests_used, api_requests_remaining

    # If no specific markets are selected, use the global markets
    if selected_markets is None:
        selected_markets = markets

    # Convert the list of markets to a comma-separated string
    market_str = ",".join(selected_markets)

    print(f"Fetching odds for sport: {sport} with markets: {market_str}")

    url = f"https://api.the-odds-api.com/v4/sports/{sport}/odds/?apiKey=3bb461e265676a9c7434e06283b69457&regions=us&markets={market_str}&oddsFormat=american" #7fa2183601e4ace7e50d6e5966f7cb6b  /  2c6684b5eb487d4e9e90bfbbffc2d903 / 3bb461e265676a9c7434e06283b69457
    response = requests.get(url)

    if response.status_code != 200:
        print(f"Error: {response.content}")
        return pd.DataFrame()  # Return an empty DataFrame if there's an error
    
    api_requests_used = int(response.headers.get('x-requests-used', '0'))
    api_requests_remaining = int(response.headers.get('x-requests-remaining', '0'))
    
    # Update the API quota label
    api_quota_label.config(text=f"API Quota: {api_requests_used} used, {api_requests_remaining} remaining")
    
    result = response.json()
    all_matches = []
    for res in result:
        for books in res['bookmakers']:
            # Skip the filtering logic if 'All' is selected
            if selected_bookmakers != 'All' and books['title'] not in selected_bookmakers:
                continue
            
            for market in books['markets']:
                match = {
                    'event_time': datetime.fromisoformat(res['commence_time'].replace('Z', '+00:00')).strftime('%Y-%m-%d %H:%M:%S'),
                    'bookmaker': books['title'],
                    'Fighter': market['outcomes'][0]['name'],
                    'Opponent': market['outcomes'][1]['name'],
                    'odds_f1': market['outcomes'][0]['price'],
                    'odds_f2': market['outcomes'][1]['price'],
                    'market': market['key']  # Add market information
                }
                all_matches.append(match)

    return pd.DataFrame(all_matches)


def download_and_calculate_all_sports(api_quota_label, selected_bookmakers='All'):
    all_arb_data = []
    for sport_key in sport_name_mapping.values():
        # Download odds for each sport
        sport_df = download_odds(sport_key, api_quota_label, selected_bookmakers)
        
        # Make sure the DataFrame has the necessary columns
        required_columns = ['event_time', 'bookmaker', 'Fighter', 'Opponent', 'odds_f1', 'odds_f2']
        for col in required_columns:
            if col not in sport_df.columns:
                sport_df[col] = None  # Add missing columns and fill with None
        
        # Calculate arbitrage opportunities
        arb_df = calculate_arbitrage(sport_df)
        
        # Add a column to indicate the sport
        arb_df['Sport'] = sport_key
        
        all_arb_data.append(arb_df)
        
    # Combine all sports data into one DataFrame
    combined_df = pd.concat(all_arb_data, ignore_index=True)
    
    # Sort by highest arbitrage percentage
    sorted_df = combined_df.sort_values('arb_percent', ascending=False)
    
    return sorted_df


# Function to calculate arbitrage opportunities
def calculate_arbitrage(df):
    arb_rows = []
    
    # Check if the DataFrame is empty or not
    if df.empty:
        return pd.DataFrame(arb_rows)
    
    # Determine the correct column name for 'bookmaker'
    if 'bookmaker' in df.columns:
        bookmaker_col_name = 'bookmaker'
    elif 'bookmaker_f1' in df.columns:
        bookmaker_col_name = 'bookmaker_f1'
    else:
        print("No suitable bookmaker column found.")
        return pd.DataFrame()

    # Group by event, fighters, and market type
    for (event_time, Fighter, Opponent, market_type), event_df in df.groupby(['event_time', 'Fighter', 'Opponent', 'market']):
        max_arb_percent = -1  # Initialize with a value that will always be less than any arb_percent
        best_arb_row = None  # Initialize with None
        
        combinations = product(event_df.itertuples(index=False), repeat=2)
        
        for row1, row2 in combinations:
            if getattr(row1, bookmaker_col_name) == getattr(row2, bookmaker_col_name):
                continue
            
            # For spreads, make sure we are considering opposing sides of the spread
            if market_type == 'spreads' and 'spread_f1' in df.columns and 'spread_f2' in df.columns:
                if row1.spread_f1 == row2.spread_f1 or row1.spread_f2 == row2.spread_f2:
                    continue
                
                # Ensure the spreads are opposite sides
                if row1.spread_f1 * row2.spread_f2 > 0:
                    continue
            
            arb_percent = round((1 / row1.odds_f1 + 1 / row2.odds_f2) * 100, 2)
            
            # Only update if this combination gives a better (i.e., smaller) arb_percent
            if arb_percent < 100 and arb_percent > max_arb_percent:
                max_arb_percent = arb_percent
                best_arb_row = {
                    'arb_percent': arb_percent,
                    'event_time': event_time,
                    'Fighter': Fighter,
                    'odds_f1': row1.odds_f1,
                    'Opponent': Opponent,
                    'odds_f2': row2.odds_f2,
                    'bookmaker_f1': getattr(row1, bookmaker_col_name),
                    'bookmaker_f2': getattr(row2, bookmaker_col_name),
                    'market': market_type  # Add market information
                }
        
        if best_arb_row:
            arb_rows.append(best_arb_row)
                
    return pd.DataFrame(arb_rows)


def resize_columns(tree):
    """Automatically resize columns based on their content."""
    for col in tree["columns"]:
        col_widths = [tree.bbox(item, col)[2] for item in tree.get_children() if tree.bbox(item, col)]
        if col_widths:  # Check if the list is not empty
            max_col_width = max(col_widths)
            tree.column(col, width=max_col_width + 10)  # Add some padding

def sortby(tree, col, descending):
    """Sort tree contents when a column is clicked on."""
    # Grab values to sort
    data = [(tree.set(child, col), child) for child in tree.get_children('')]
    
    # Reorder data
    data.sort(reverse=descending)
    for indx, item in enumerate(data):
        tree.move(item[1], '', indx)
    
    # Switch the heading so that it will sort in the opposite direction
    tree.heading(col,
                 command=lambda col=col: sortby(tree, col, int(not descending)))
    

def populate_table(tree, df, is_upcoming_tab=False):
    # Clear existing rows
    for row in tree.get_children():
        tree.delete(row)
    
    if df.empty:
        print("DataFrame is empty.")
        return

    # Check if DataFrame has 'arb_percent' and 'market' columns
    has_arb_percent = 'arb_percent' in df.columns
    has_market = 'market' in df.columns

    for index, row in df.iterrows():
        values_to_insert = [
            row['arb_percent'] if has_arb_percent else 'N/A',  # Use 'N/A' if 'arb_percent' is missing
            row['event_time'],
            row['Fighter'],
            row['odds_f1'],
            row.get('bookmaker_f1', row.get('bookmaker', 'N/A')),  # Adapt for missing 'bookmaker_f1'
            row['Opponent'],
            row['odds_f2'],
            row.get('bookmaker_f2', row.get('bookmaker', 'N/A')),  # Adapt for missing 'bookmaker_f2'
            row['market'] if has_market else 'N/A'  # Use 'N/A' if 'market' is missing
        ]
        
        if is_upcoming_tab:
            values_to_insert.insert(2, row.get('Sport', 'N/A'))
        
        tree.insert('', 'end', values=values_to_insert)

    # Call the function to auto-size columns
    resize_columns(tree)

# Initialize a global lock for managing cache and other shared resources
cache_lock = Lock()

# Function to refresh the data and table
def refresh_table(tree, sport, api_quota_label, bookmaker_listbox=None, is_upcoming_tab=False, force_api_call=False):
    global last_update_time
    
    current_time = time.time()
    
    # Determine the appropriate cache file based on whether this is the "Upcoming" tab
    if is_upcoming_tab:
        cache_file = 'upcoming_arbitrage_data.csv'
    else:
        cache_file = f'arbitrage_data_{sport}.csv'
    
    # Determine selected bookmakers based on listbox selection
    selected_bookmakers = 'All'
    if bookmaker_listbox is not None:
        selected_indices = bookmaker_listbox.curselection()
        selected_bookmakers = [bookmaker_listbox.get(i) for i in selected_indices]
        if "All" in selected_bookmakers or not selected_bookmakers:
            selected_bookmakers = 'All'
    
    # Skip API call if sport is None (for the Upcoming tab)
    if sport is None and not is_upcoming_tab:
        return

    # If force_api_call is True, ignore the cache and fetch new data
    if force_api_call or not os.path.exists(cache_file) or (current_time - last_update_time.get(sport, 0)) >= CACHE_EXPIRY_TIME:
        if is_upcoming_tab:
            df = download_and_calculate_all_sports(api_quota_label, selected_bookmakers)  # <- Used here
        else:
            df = download_odds(sport, api_quota_label, selected_bookmakers)
        
        # Save to cache file if DataFrame is not empty
        if not df.empty:
            df.to_csv(cache_file, index=False)
        
        # Update the last update time for this sport or "Upcoming" tab
        last_update_time[sport] = current_time
    else:
        # If the cache exists and has not expired, read from the cache file
        df = pd.read_csv(cache_file)


   # DataFrame check and population logic
    if df.empty:
        print("No data available for selected filters.")
        populate_table(tree, pd.DataFrame())  # Pass an empty DataFrame
    else:
        if is_upcoming_tab:  # If it's the "Upcoming" tab, use the combined DataFrame directly
            populate_table(tree, df, is_upcoming_tab=True)
        else:
            arb_df = calculate_arbitrage(df)
            populate_table(tree, arb_df)
    
    # Validate DataFrame columns and adapt to available ones
    bookmaker_cols = [col for col in ['bookmaker_f1', 'bookmaker_f2', 'bookmaker'] if col in df.columns]
    
    if not bookmaker_cols:
        print(f"DataFrame columns: {df.columns}")
        print("No suitable bookmaker column found. Skipping filter.")
        populate_table(tree, df)
        return

    # Apply bookmaker filter to DataFrame
    selected_bookmakers = 'All'
    if bookmaker_listbox is not None:
        selected_indices = bookmaker_listbox.curselection()
        selected_bookmakers = [bookmaker_listbox.get(i) for i in selected_indices]
        if "All" in selected_bookmakers or not selected_bookmakers:
            selected_bookmakers = 'All'
    
    if selected_bookmakers != 'All':
        # Use the available bookmaker columns for filtering
        df = df[df[bookmaker_cols].isin(selected_bookmakers).any(axis=1)]


    # Update the API quota label to include the last refresh time
    last_update_str = datetime.fromtimestamp(last_update_time.get(sport, current_time)).strftime('%Y-%m-%d %H:%M:%S')
    api_quota_label.config(text=f"API Quota: {api_requests_used} used, {api_requests_remaining} remaining | Last refresh: {last_update_str}")

    # Populate the table; if DataFrame is empty, this will clear the table
    if df.empty:
        print("No data available for selected filters.")
        populate_table(tree, pd.DataFrame())
    else:
        if is_upcoming_tab:  # If it's the "Upcoming" tab, use the combined DataFrame directly
            populate_table(tree, df, is_upcoming_tab=True)
        else:
            arb_df = calculate_arbitrage(df)
            populate_table(tree, arb_df)


# Function to initialize the table with cached data
def initialize_table(tree, sport_key, api_quota_label, is_upcoming_tab=False):
    global last_update_time
    
    # Determine the appropriate cache file based on whether this is the "Upcoming" tab
    if is_upcoming_tab:
        cache_file = 'upcoming_arbitrage_data.csv'
    else:
        cache_file = f'arbitrage_data_{sport_key}.csv'
    
    current_time = time.time()
    
    if os.path.exists(cache_file):
        df = pd.read_csv(cache_file)
        
        # Update the API quota label to include last refresh time
        last_update_str = datetime.fromtimestamp(last_update_time.get(sport_key, current_time)).strftime('%Y-%m-%d %H:%M:%S')
        api_quota_label.config(text=f"API Quota: {api_requests_used} used, {api_requests_remaining} remaining | Last refresh: {last_update_str}")
        
        # If it's the "Upcoming" tab, use the combined DataFrame directly
        if is_upcoming_tab:
            populate_table(tree, df, is_upcoming_tab=True)
        else:
            arb_df = calculate_arbitrage(df)
            populate_table(tree, arb_df)
    else:
        api_quota_label.config(text=f"No cached data. Please manually refresh.")


# Create a mapping from human-readable names to sport keys
sport_name_mapping = {
    'UFC': 'mma_mixed_martial_arts',
    'BOXING': 'boxing_boxing',
    'NFL': 'americanfootball_nfl',
    'NCAAF': 'americanfootball_ncaaf',
    'MLB': 'baseball_mlb',
    'NBA': 'basketball_nba',
    'NHL': 'icehockey_nhl'
}

def create_sport_tab(tab_control, sport_key, human_readable_name, api_quota_label, is_upcoming_tab=False, auto_refresh_time=0):
    # Create a new tab
    tab = ttk.Frame(tab_control)
    tab_control.add(tab, text=human_readable_name)  # Use the human-readable name for the tab
    
    # Create a listbox for selecting bookmakers
    bookmaker_listbox = Listbox(tab, selectmode=MULTIPLE)
    bookmaker_listbox.insert('end', 'All')
    for bookmaker in known_bookmakers:
        bookmaker_listbox.insert('end', bookmaker)
    bookmaker_listbox.pack(side='right', fill='y')
    
    # Define columns, adding 'Sport' conditionally if it's the "Upcoming" tab
    columns = ['Arbitrage %', 'Event Time', 'Player', 'Best Odds P1', 'Bookmaker P1', 'Opponent', 'Best Odds P2', 'Bookmaker P2', 'Market']
    if is_upcoming_tab:
        columns.insert(2, 'Sport')  # Insert 'Sport' at the third position if it's the "Upcoming" tab

    # Initialize the Treeview with the columns
    tree = ttk.Treeview(tab, columns=columns, show='headings')
    
    for col in columns:
        tree.heading(col, text=col, 
                     command=lambda _col=col: sortby(tree, _col, 0))
        tree.column(col, width=100)

    tree.pack(side='left', fill='both', expand=True)
    
    # Initialize table with any cached data available
    initialize_table(tree, sport_key, api_quota_label, is_upcoming_tab=is_upcoming_tab)
    
    # Add a scrollbar for the Treeview
    scrollbar = ttk.Scrollbar(tab, orient='vertical', command=tree.yview)
    scrollbar.pack(side='right', fill='y')
    tree.configure(yscrollcommand=scrollbar.set)
    
    # Add a button for manual refresh
    btn_refresh = ttk.Button(tab, text="Manual Refresh", command=lambda: refresh_table(tree, sport_key, api_quota_label, bookmaker_listbox, is_upcoming_tab, force_api_call=True))
    btn_refresh.pack(side='bottom')
    
    # Add a button to apply filter
    btn_apply_filter = ttk.Button(tab, text="Apply Filter", command=lambda: refresh_table(tree, sport_key, api_quota_label, bookmaker_listbox, is_upcoming_tab, True))
    btn_apply_filter.pack(side='bottom')

    # Periodic refresh function
    def periodic_refresh():
        # Update selected bookmakers based on listbox selection
        selected_indices = bookmaker_listbox.curselection()
        selected_bookmakers = [bookmaker_listbox.get(i) for i in selected_indices]
        if "All" in selected_bookmakers or not selected_bookmakers:
            selected_bookmakers = 'All'

        # Only call download_and_calculate_all_sports for the "Upcoming" tab
        if is_upcoming_tab:
            threading.Thread(target=download_and_calculate_all_sports, args=(api_quota_label, selected_bookmakers)).start()
        else:
            threading.Thread(target=refresh_table, args=(tree, sport_key, api_quota_label, bookmaker_listbox, is_upcoming_tab)).start()
        
        root.after(auto_refresh_time, periodic_refresh)
    
    # Start periodic refresh if a time interval is set
    if auto_refresh_time:
        periodic_refresh()

# Tkinter GUI setup
root = tk.Tk()
root.title("Live Arbitrage Opportunities")

# Label to display API quota
api_quota_label = ttk.Label(root, text=f"API Quota: {api_requests_used} used, {api_requests_remaining} remaining")
api_quota_label.pack(side='bottom')

# Create a tab control
tab_control = ttk.Notebook(root)
tab_control.pack(expand=1, fill='both')

# Create tabs dynamically for individual sports
for human_readable_name, sport_key in sport_name_mapping.items():
    create_sport_tab(tab_control, sport_key, human_readable_name, api_quota_label, auto_refresh_time=0)

# Create the "Upcoming" tab
create_sport_tab(tab_control, None, 'Upcoming', api_quota_label, is_upcoming_tab=True, auto_refresh_time=0)

# Tkinter event loop
root.mainloop()